---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **`ConversationSummaryMemory`**

* **시간의 경과에 따른 대화의 요약 생성 = `길었던 대화를 요약`해서 `저장`하는 메모리**

* `시간 경과`에 따른 대화의 **`정보`를 압축하는 데 유용**

  * 대화 요약 메모리는 대화가 진행되는 동안 `대화`를 `요약`하고 `현재 요약`을 `메모리에 저장`

  * → 지금까지의 **`대화 요약`을 `프롬프트`/`체인`에 삽입 가능**

  * 따라서, 과거 메시지 기록을 프롬프트에 그대로 보관하면 토큰을 너무 많이 차지할 수 있는 **`긴 대화`에 `가장 유용`**

<br>

* **역할**: 

    * 모든 대화 내용을 기억하는 대신, **`핵심 내용만 간결하게 요약하여 저장`**

    * **`≒ 길었던 회의 내용을 간단한 요약본으로 정리한 것`**

In [1]:
# 환경변수 처리 및 클라이언트 생성
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

import os
import json

# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [ ]:
# LangSmith 추적 설정하기 (https:smith.langchin.com)
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable                                                             # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음"      # API 키 값은 직접 출력하지 않음
org = "설정됨" if os.getenv('LANGCHAIN_ORGANIZATION') else "설정되지 않음"                      # 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")


<small>

* 셀 출력

    ```markdown
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

---

* 대화 요약 메모리 모듈 추출을 위한 임포트

In [3]:
from langchain.memory import ConversationSummaryMemory

---

* **`LLM` 생성하기**

In [ ]:
import os
from dotenv import load_dotenv
import openai

# .env 파일에서 환경변수 불러오기
load_dotenv()

# 환경변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI API 키 설정
openai.api_key = api_key

# OpenAI를 불러오기
# ✅ 디버깅 함수: API 키가 잘 불러와졌는지 확인
def debug_api_key():
    if api_key is None:
        print("❌ API 키를 불러오지 못했습니다. .env 파일과 변수명을 확인하세요.")
    elif api_key.startswith("sk-") and len(api_key) > 20:
        print("✅ API 키를 성공적으로 불러왔습니다.")
    else:
        print("⚠️ API 키 형식이 올바르지 않은 것 같습니다. 값을 확인하세요.")

# 디버깅 함수 실행
debug_api_key()

<small>

* 셀 출력

    ```markdown
    ✅ API 키를 성공적으로 불러왔습니다.
    ```

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

memory = ConversationSummaryMemory(
    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        model="gpt-4o-mini",    
        ),
    return_messages=True)

<small>

* 셀 출력 (0.3s)

    ```markdown
    /var/folders/h3/l7wnkv352kqftv0t8ctl2ld40000gn/T/ipykernel_12653/1215817412.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
    memory = ConversationSummaryMemory(
    ```

<br>

---

<br>

* 셀 출력 의미
  
  * `ConversationSummaryMemory`는 `LangChain v0.3.1`부터 **사용 중단 예정(`deprecated`) → `langchain==1.0.0 버전`에서 `완전히 제거`**

    * 즉, 이 모듈은 더이상 추천되지 않음 → **`migration 가이드` 참고할 것**

    * 이유: **`LangChain`은 현재 메모리 시스템이 더 유연한 `LangGraph`, `LCEL` 방식으로 `업그레이드` 중이기 때문**

  * 공식 가이드
    * [마이그레이션 안내 가이드](https://python.langchain.com/docs/versions/migrating_memory/)
    * [해당 메모리 마이그레이션 안내 가이드](https://python.langchain.com/docs/versions/migrating_memory/conversation_summary_memory/)

<br>

---

<br>

* **`최신 코드 migration` 방법**

  * 기존의 `ConversationSummaryMemory` 대신 **`LangGraph` 사용 → 대화 요약 메모리 구현 가능**

  * 단계

    * 1) `LangGraph` 설치: `pip install langgraph`

    * 2) 요약 로직 = `LangGraph`에 통합

    * 3) **`trim_messages`로 토큰 제한 관리 = 기존의 `ConversationSummaryMemory` 대체**
  
  * `test`에 최신 코드 시도 예정 
  

---

* 여러 대화 저장하기

In [6]:
memory.save_context(
    inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
    outputs={
        "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
    },
)
memory.save_context(
    inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
    outputs={
        "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "여행자 보험은 포함되어 있나요?"},
    outputs={
        "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
    },
)
memory.save_context(
    inputs={
        "human": "항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?"
    },
    outputs={
        "ai": "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
    },
)
memory.save_context(
    inputs={"human": "패키지에 포함된 호텔의 등급은 어떻게 되나요?"},
    outputs={
        "ai": "이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다."
    },
)
memory.save_context(
    inputs={"human": "식사 옵션에 대해 더 자세히 알려주실 수 있나요?"},
    outputs={
        "ai": "이 여행 패키지는 매일 아침 호텔에서 제공되는 조식을 포함하고 있습니다. 점심과 저녁 식사는 포함되어 있지 않아, 여행자가 자유롭게 현지의 다양한 음식을 경험할 수 있는 기회를 제공합니다. 또한, 각 도시별로 추천 식당 리스트를 제공하여 현지의 맛을 최대한 즐길 수 있도록 도와드립니다."
    },
)
memory.save_context(
    inputs={"human": "패키지 예약 시 예약금은 얼마인가요? 취소 정책은 어떻게 되나요?"},
    outputs={
        "ai": "패키지 예약 시 500유로의 예약금이 필요합니다. 취소 정책은 예약일로부터 30일 전까지는 전액 환불이 가능하며, 이후 취소 시에는 예약금이 환불되지 않습니다. 여행 시작일로부터 14일 전 취소 시 50%의 비용이 청구되며, 그 이후는 전액 비용이 청구됩니다."
    },
)


<small>

* 처리 시간 (1m 34.5s)

In [ ]:
print(type(memory.load_memory_variables({})["history"]))            # <class 'list'>

In [ ]:
# 저장된 메모리 확인

print(memory.load_memory_variables({})["history"])

<small>

* 셀 출력_(첫번째)

    ```python
    [SystemMessage(content='The human asks about the price of a European travel package. The AI responds that the basic price for a 14-night, 15-day package is 3,500 euros, which includes airfare, hotel accommodations, and entrance fees to designated tourist sites, with additional costs depending on selected optional tours or personal expenses. The human then inquires about the main tourist attractions included in the trip, and the AI lists famous sites such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and the Rhine Falls in Zurich, highlighting that travelers will experience the representative landmarks of each city. The human further asks if traveler insurance is included, and the AI confirms that basic traveler insurance is provided, covering medical expenses and emergency support, with options for additional coverage available. The human then asks if it is possible to upgrade to business class for the flights and inquires about the cost, to which the AI confirms that an upgrade is possible for an additional 1,200 euros round trip, offering benefits such as wider seats, better in-flight meals, and increased luggage allowance. The human also asks about the rating of the hotels included in the package, and the AI replies that the package includes stays at 4-star hotels, which provide comfort and convenience, are centrally located for easy access to tourist sites, and offer excellent service and amenities. Finally, the human asks for more details about meal options, and the AI explains that the package includes daily breakfast at the hotel, while lunch and dinner are not included, allowing travelers to experience local cuisine freely, and that a list of recommended restaurants in each city will be provided to enhance their culinary experience. The human then inquires about the reservation deposit and cancellation policy, to which the AI states that a deposit of 500 euros is required at the time of booking, and that cancellations made 30 days prior to the reservation date are fully refundable, while cancellations made within 14 days of the travel date incur a 50% charge, and later cancellations are non-refundable.', additional_kwargs={}, response_metadata={})]
    ```

<br>

* 셀 출력_(ver_ko) = `content 내용을 한국어로 해석`

    ```python
    [SystemMessage(content='고객이 유럽 여행 패키지의 가격을 묻자, AI는 14박 15일 일정의 기본 가격이 3,500유로이며 항공료, 호텔 숙박비, 지정 관광지 입장료가 포함되어 있고, 선택 투어나 개인 경비에 따라 추가 비용이 발생한다고 답했습니다. 고객이 여행에 포함된 주요 관광 명소를 묻자, AI는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라인 폭포 등 유명한 명소를 나열하며, 각 도시를 대표하는 랜드마크를 경험할 수 있다고 강조했습니다. 이어 고객이 여행자 보험 포함 여부를 묻자, AI는 기본 여행자 보험이 제공되며 의료비와 긴급 지원을 보장하고, 추가 보장 옵션도 선택할 수 있다고 확인했습니다. 고객이 항공편을 비즈니스 클래스로 업그레이드할 수 있는지와 비용을 묻자, AI는 왕복 기준으로 추가 1,200유로를 지불하면 업그레이드가 가능하며, 넓은 좌석, 향상된 기내식, 수하물 허용량 증가 등의 혜택이 제공된다고 답했습니다. 또 고객이 포함된 호텔 등급을 묻자, AI는 이 패키지에 4성급 호텔 숙박이 포함되어 있어 편안하고 편리하며 관광지 접근성이 좋은 도심에 위치하고, 뛰어난 서비스와 편의시설을 제공한다고 설명했습니다. 마지막으로 고객이 식사 옵션에 대한 자세한 내용을 요청하자, AI는 매일 호텔 조식이 포함되어 있고 중식과 석식은 불포함이라 여행자가 자유롭게 현지 음식을 즐길 수 있으며, 각 도시별 추천 레스토랑 목록을 제공해 미식 경험을 더욱 풍성하게 해준다고 안내했습니다. 이어 예약 보증금 및 취소 정책을 묻자, AI는 예약 시 500유로의 보증금이 필요하며, 여행일 30일 전까지 취소하면 전액 환불되지만 여행일 14일 이내 취소 시에는 50% 수수료가 부과되며 그 이후에는 환불이 불가능하다고 안내했습니다.', additional_kwargs={}, response_metadata={})]
    ```

<br>

* 셀 출력_2 - 메모리가 꼬여서 두번째로 출력된 내용

    ```python
    [HumanMessage(content='유럽 여행 패키지의 가격은 얼마인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='여행 중에 방문할 주요 관광지는 어디인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='여행자 보험은 포함되어 있나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='패키지에 포함된 호텔의 등급은 어떻게 되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='식사 옵션에 대해 더 자세히 알려주실 수 있나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='이 여행 패키지는 매일 아침 호텔에서 제공되는 조식을 포함하고 있습니다. 점심과 저녁 식사는 포함되어 있지 않아, 여행자가 자유롭게 현지의 다양한 음식을 경험할 수 있는 기회를 제공합니다. 또한, 각 도시별로 추천 식당 리스트를 제공하여 현지의 맛을 최대한 즐길 수 있도록 도와드립니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='패키지 예약 시 예약금은 얼마인가요? 취소 정책은 어떻게 되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='패키지 예약 시 500유로의 예약금이 필요합니다. 취소 정책은 예약일로부터 30일 전까지는 전액 환불이 가능하며, 이후 취소 시에는 예약금이 환불되지 않습니다. 여행 시작일로부터 14일 전 취소 시 50%의 비용이 청구되며, 그 이후는 전액 비용이 청구됩니다.', additional_kwargs={}, response_metadata={})]
    ```

<br>

  ---

<br>

* `memory.load_memory_variables({})["history"]`의 출력값은 `SystemMessage 객체`를 포함한 `리스트`이며, `SystemMessage.content`에 실제 텍스트가 들어있음

In [ ]:
# 가독성을 높인 출력

from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory

history = memory.load_memory_variables({})["history"]

for i, msg in enumerate(history, start=1):
    print("="*50)
    print(f"메시지 {i} - 타입: {msg.__class__.__name__}")
    print(msg.content)
    print()

<small>

* 셀 출력_1

    ```markdown
    ==================================================
    메시지 1 - 타입: SystemMessage
    The human asks about the price of a European travel package. The AI responds that the basic price for a 14-night, 15-day package is 3,500 euros, which includes airfare, hotel accommodations, and entrance fees to designated tourist sites, with additional costs depending on selected optional tours or personal expenses. The human then inquires about the main tourist attractions included in the trip, and the AI lists famous sites such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and the Rhine Falls in Zurich, highlighting that travelers will experience the representative landmarks of each city. The human further asks if traveler insurance is included, and the AI confirms that basic traveler insurance is provided, covering medical expenses and emergency support, with options for additional coverage available. The human then asks if it is possible to upgrade to business class for the flights and inquires about the cost, to which the AI confirms that an upgrade is possible for an additional 1,200 euros round trip, offering benefits such as wider seats, better in-flight meals, and increased luggage allowance. The human also asks about the rating of the hotels included in the package, and the AI replies that the package includes stays at 4-star hotels, which provide comfort and convenience, are centrally located for easy access to tourist sites, and offer excellent service and amenities. The human then asks for more details about meal options, and the AI explains that the package includes daily breakfast at the hotel, while lunch and dinner are not included, allowing travelers to experience local cuisine freely, and that a list of recommended restaurants in each city will be provided to enhance their culinary experience. Finally, the human inquires about the reservation deposit and cancellation policy, to which the AI responds that a deposit of 500 euros is required, and full refunds are available for cancellations made 30 days prior to the reservation date, while later cancellations incur fees, including a 50% charge if canceled 14 days before the trip and full charges thereafter.
    ```

<br>

  ---

<br>

* 셀 출력_2 - 메모리가 꼬여서 두번째로 출력된 내용

    ```markdown
    ==================================================
    메시지 1 - 타입: HumanMessage
    유럽 여행 패키지의 가격은 얼마인가요?

    ==================================================
    메시지 2 - 타입: AIMessage
    유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다.

    ==================================================
    메시지 3 - 타입: HumanMessage
    여행 중에 방문할 주요 관광지는 어디인가요?

    ==================================================
    메시지 4 - 타입: AIMessage
    이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다.

    ==================================================
    메시지 5 - 타입: HumanMessage
    여행자 보험은 포함되어 있나요?

    ==================================================
    메시지 6 - 타입: AIMessage
    네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다.

    ==================================================
    메시지 7 - 타입: HumanMessage
    항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?

    ==================================================
    메시지 8 - 타입: AIMessage
    항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다.

    ==================================================
    메시지 9 - 타입: HumanMessage
    패키지에 포함된 호텔의 등급은 어떻게 되나요?

    ==================================================
    메시지 10 - 타입: AIMessage
    이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다.

    ==================================================
    메시지 11 - 타입: HumanMessage
    식사 옵션에 대해 더 자세히 알려주실 수 있나요?

    ==================================================
    메시지 12 - 타입: AIMessage
    이 여행 패키지는 매일 아침 호텔에서 제공되는 조식을 포함하고 있습니다. 점심과 저녁 식사는 포함되어 있지 않아, 여행자가 자유롭게 현지의 다양한 음식을 경험할 수 있는 기회를 제공합니다. 또한, 각 도시별로 추천 식당 리스트를 제공하여 현지의 맛을 최대한 즐길 수 있도록 도와드립니다.

    ==================================================
    메시지 13 - 타입: HumanMessage
    패키지 예약 시 예약금은 얼마인가요? 취소 정책은 어떻게 되나요?

    ==================================================
    메시지 14 - 타입: AIMessage
    패키지 예약 시 500유로의 예약금이 필요합니다. 취소 정책은 예약일로부터 30일 전까지는 전액 환불이 가능하며, 이후 취소 시에는 예약금이 환불되지 않습니다. 여행 시작일로부터 14일 전 취소 시 50%의 비용이 청구되며, 그 이후는 전액 비용이 청구됩니다.
    ```

---

### **`ConversationSummeryBufferMemory`**

* **`두 가지 아이디어`를 `결합`한 것**

  * **`최근 대화내용`의 버퍼를 메모리에 `유지`**

  * **`이전 대화 내용`을 완전히 플러시(`flush`)하지 않고 `요약으로 컴파일`** → 두 가지를 모두 사용

<br>

* **`대화 내용`** 을 플러시할 시기를 결정하기 위해 상호작용의 개수가 아닌 **`토큰 길이`** 사용

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o-mini",
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=200,                                    # 요약의 기준이 되는 토큰 길이 설정
    return_messages=True,
)

<small>

* 셀 출력

    ```python
    /var/folders/h3/l7wnkv352kqftv0t8ctl2ld40000gn/T/ipykernel_12653/3116438127.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
    memory = ConversationSummaryBufferMemory(
    ```

---

* 먼저 **`1개의 대화만 저장`** → 메모리 확인해보기

In [11]:
memory.save_context(
    inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
    outputs={
        "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
    },
)

<small>

* 처리 시간 (2.6s)

---

* 메모리에 저장된 대화 확인해보기

  * **아직은 대화 내용을 요약하지 않음 → 기준이 되는 `200 토큰에 도달하지 않았기 때문`**

In [ ]:
# 메모리에 저장된 대화내용 확인

memory.load_memory_variables({})["history"]

<small>

* 셀 출력

    ```python
    [HumanMessage(content='유럽 여행 패키지의 가격은 얼마인가요?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다.', additional_kwargs={}, response_metadata={})]
    ```

---

* **`대화 추가` 저장** = **`200 토큰 제한 넘김`**

In [13]:
# 대화 추가하기
memory.save_context(
    inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
    outputs={
        "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "여행자 보험은 포함되어 있나요?"},
    outputs={
        "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
    },
)
memory.save_context(
    inputs={
        "human": "항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?"
    },
    outputs={
        "ai": "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
    },
)
memory.save_context(
    inputs={"human": "패키지에 포함된 호텔의 등급은 어떻게 되나요?"},
    outputs={
        "ai": "이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다."
    },
)

<small>

* 처리 시간 (8.7s)

---

* 저장된 내용 확인하기

  * **`가장 최근 1개의 대화 ≠ 요약`**

  * **`이전`의 대화 내용 = `요약본`으로 저장**

In [ ]:
# 메모리에 저장된 대화내용 확인

memory.load_memory_variables({})["history"]

<small>

* 셀 출력

    ```python
    [SystemMessage(content='The human asks about the price of a European travel package. The AI responds that the basic price for a 14-night, 15-day package is 3,500 euros, which includes airfare, hotel accommodations, and entrance fees to designated tourist sites, with additional costs depending on selected optional tours or personal expenses. The human then inquires about the main tourist attractions included in the trip, and the AI lists famous sites such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and the Rhine Falls in Zurich, highlighting that travelers will experience the iconic landmarks of each city. The human further asks if traveler insurance is included, and the AI confirms that basic traveler insurance is provided, covering medical expenses and emergency support, with options for additional coverage available.', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다.', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='패키지에 포함된 호텔의 등급은 어떻게 되나요?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다.', additional_kwargs={}, response_metadata={})]
    ```

In [ ]:
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory

history = memory.load_memory_variables({})["history"]

for i, msg in enumerate(history, start=1):
    print("="*50)
    print(f"메시지 {i} - 타입: {msg.__class__.__name__}")
    print(msg.content)
    print()

<small>

* 셀 출력

    ```markdown
    ==================================================
    메시지 1 - 타입: SystemMessage
    The human asks about the price of a European travel package. The AI responds that the basic price for a 14-night, 15-day package is 3,500 euros, which includes airfare, hotel accommodations, and entrance fees to designated tourist sites, with additional costs depending on selected optional tours or personal expenses. The human then inquires about the main tourist attractions included in the trip, and the AI lists famous sites such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and the Rhine Falls in Zurich, highlighting that travelers will experience the iconic landmarks of each city. The human further asks if traveler insurance is included, and the AI confirms that basic traveler insurance is provided, covering medical expenses and emergency support, with options for additional coverage available.

    ==================================================
    메시지 2 - 타입: HumanMessage
    항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?

    ==================================================
    메시지 3 - 타입: AIMessage
    항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다.

    ==================================================
    메시지 4 - 타입: HumanMessage
    패키지에 포함된 호텔의 등급은 어떻게 되나요?

    ==================================================
    메시지 5 - 타입: AIMessage
    이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다.
    ```

---

### **`업데이트된 코드`**

* *공식 가이드 사이트*

  * [migration 가이드](https://python.langchain.com/docs/versions/migrating_memory/conversation_summary_memory/): ConversationSummaryMemory migration 상세 설명
  
  * [LangGraph 메모리 how-to](https://python.langchain.com/docs/how_to/chatbots_memory/): 대화봇 메모리 최신 튜토리얼
  
  * [API 레퍼런스](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.summary.ConversationSummaryMemory.html): deprecation 상세 및 대체 방법

---

* 최신 코드로 시도해보기

In [191]:
# 필요한 모듈 임포트 (최신 LangChain에서 메모리 관리)
from typing import TypedDict
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, RemoveMessage, trim_messages
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langsmith import traceable
import uuid                                                             # 세션 ID 생성용

In [ ]:
# 필요한 모듈 임포트
from dotenv import load_dotenv
import os
import json

# 랭스미스 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

# LangSmith 환경 변수 확인
print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음"      # API 키 값은 직접 출력하지 않음
org = "설정됨" if os.getenv('LANGCHAIN_ORGANIZATION') else "설정되지 않음"                      # 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

In [193]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI

# .env 파일에서 환경변수 불러오기
load_dotenv()

# 환경변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI API 키 설정
openai.api_key = api_key

# 1_LLM 초기화
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o-mini",
    openai_api_key=api_key,
)

In [194]:
memory = MemorySaver()

In [195]:
# State 확장 (기본 MessagesState에 summary 필드 추가 – 요약 저장용)

class State(TypedDict):
    messages: list                  # 대화 메시지 리스트 (기본 제공)
    summary: str                    # 요약 텍스트 저장 필드

In [196]:
# StateGraph 생성

workflow = StateGraph(state_schema=State)

In [197]:
# 요약 생성 함수 (기존 요약 + 새 메시지로 LLM 호출해 업데이트)
def update_summary(state: State):
    existing_summary = state.get("summary", "")             # 기존 요약 가져오기 (없으면 빈 문자열)
    messages = state["messages"]                            # 현재 대화 메시지 리스트

    # 요약 프롬프트 템플릿 (기존 요약 기반으로 새 요약 생성)
    #summary_prompt_text = f"""
    #Conversation summary so far: {existing_summary}
    #Extend the summary by taking into account these new messages:
    #"""

    #prompt = ChatPromptTemplate.from_messages([
    #    SystemMessage(content=summary_prompt_text),         # 요약 지시
    #    *messages                                           # 새 메시지 추가
    #])
    # LLM 으로 요약 갱신
    prompt = ChatPromptTemplate.from_messages(
        [SystemMessage(content=f"Conversation summary so far:\n{existing_summary}\n\nUpdate it."),
         *messages]
    )
    new_summary = (prompt | llm).invoke({}).content

    # 2) 최근 4개만 남기고 나머지 삭제
    keep_messages   = messages[-4:]              # 남길 메시지
    delete_messages = [RemoveMessage(id=m.id)    # 지울 메시지
                        for m in messages[:-4]]

    # LLM 호출로 새 요약 생성
    chain = prompt | llm
    summary_response = chain.invoke({})

    # 오래된 메시지 삭제 (최근 2개만 유지 – 필요 시 조정)
    #delete_messages = []
    #if len(messages) > 4:                                   # 4개로 늘려 최근 버퍼 보호
    #    delete_messages = [RemoveMessage(id=m.id) for m in messages[:-4]]  # 최근 4개 제외 삭제

    # 업데이트된 요약과 삭제 메시지 반환
    #return {"summary": summary_response.content, "messages": delete_messages}
    
    # 3) **남길 메시지 + 삭제 지시**를 함께 반환
    return {
        "summary":  new_summary,
        "messages": keep_messages + delete_messages
    }


In [198]:
# 그래프 노드·엣지 등록 (START → 요약 노드)

workflow.add_edge(START, "summary")
workflow.add_node("summary", update_summary)
                                                    # <langgraph.graph.state.StateGraph at 0x119cf22c0>

In [200]:
# 메모리 설정 (MemorySaver로 체크포인트 저장)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [201]:
# 세션 ID 생성 및 config 설정
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

In [202]:
# 대화 턴
pairs = [
    ("커피 원두 종류가 뭐야?",
    "커피 원두는 크게 아라비카(Arabica), 로부스타(Robusta), 그리고 리베리카(Liberica)로 나뉘며, 각각의 품종은 고유한 맛과 향을 지니고 있습니다.  아라비카는 부드럽고 산미가 있으며 향이 풍부해 고급 커피에 주로 사용되며, 브라질이나 에티오피아 같은 고지대에서 재배됩니다.  반면 로부스타는 카페인 함량이 높고 쓴맛이 강하며 바디감이 묵직해 에스프레소 블렌드에 자주 활용되며, 베트남이나 인도네시아 같은 저지대에서 자랍니다.  마지막으로 리베리카는 생산량이 적고 독특한 나무 향과 꽃 향기를 지녀 이색적인 커피를 찾는 사람들에게 인기가 있으며, 필리핀이나 말레이시아 등에서 재배됩니다. 각 원두는 재배 환경과 가공 방식에 따라 맛이 달라지므로, 자신의 취향에 맞는 커피를 찾는 데 중요한 기준이 됩니다."),
    ("라떼 만드는 방법 알려줘.",
    "먼저 에스프레소를 한 샷 추출하는데, 이때 사용하는 원두는 아라비카처럼 부드럽고 향이 풍부한 것이 좋으며, 진한 맛을 원한다면 로부스타를 블렌딩해도 괜찮습니다.  에스프레소가 준비되면 우유를 끓기 직전까지 데운 후 거품을 내주는데, 거품기는 물론 프렌치프레스나 병에 넣고 흔드는 방법으로도 충분히 가능합니다.  그 다음 에스프레소가 담긴 컵에 스팀한 우유를 천천히 붓고, 마지막으로 부드러운 우유 거품을 숟가락으로 떠서 위에 올려주면 라떼가 완성됩니다.  취향에 따라 우유의 양을 조절하면 더 고소하거나 진한 맛을 즐길 수 있으며, 얼음을 넣고 차가운 우유와 에스프레소를 섞으면 아이스 라떼로도 즐길 수 있습니다."),
    ("커피를 마시기 좋은 시간은?",
    "커피를 마시기에 가장 좋은 시간은 아침 기상 직후가 아니라, 코르티솔 수치가 안정되는 오전 9시 30분에서 11시 30분 사이입니다.  기상 직후에는 우리 몸이 이미 각성 상태에 들어가기 위해 코르티솔이라는 스트레스 호르몬을 많이 분비하기 때문에, 이때 커피를 마시면 오히려 몸에 부담을 줄 수 있습니다.  점심 식사 직후에도 위가 팽창된 상태에서 커피를 마시면 소화에 방해가 될 수 있으므로, 점심을 먹고 최소 1시간 정도 지난 후에 마시는 것이 좋습니다.  또한 늦은 오후나 저녁 시간에 커피를 마시면 카페인의 각성 효과로 인해 수면에 영향을 줄 수 있으므로 피하는 것이 바람직합니다.  따라서 하루 중 커피를 가장 맛있고 건강하게 즐길 수 있는 시간은 오전 중 코르티솔 수치가 낮아지는 9시 30분에서 11시 30분 사이이며, 오후에는 1시 30분에서 5시 사이가 적절한 커피 타임으로 추천됩니다."),
    ("카페 추천 메뉴 알려줘.",
    "카페에서 가장 인기 있는 메뉴로는 아이스 아메리카노가 단연 첫 손에 꼽히며, 깔끔하고 시원한 맛 덕분에 계절을 가리지 않고 많은 사람들이 즐깁니다.  그 뒤를 이어 라떼는 부드러운 우유와 에스프레소의 조화로 고소한 풍미를 자랑하며, 바닐라나 헤이즐넛 시럽을 추가하면 더욱 달콤하게 즐길 수 있습니다.  요즘에는 말차 라떼나 흑임자 라떼처럼 고소하고 이색적인 재료를 활용한 메뉴도 인기를 끌고 있으며, 특히 한옥 감성 카페에서는 전통적인 재료를 현대적으로 재해석한 음료가 눈길을 끕니다.  디저트와 함께 즐기기 좋은 메뉴로는 콜드브루가 추천되며, 진한 커피 맛과 함께 피넛 버터 쿠키나 크루아상 같은 베이커리류가 잘 어울립니다.  또한 여름 시즌에는 계절 과일을 활용한 타르트나 에클레어 같은 디저트가 함께 제공되는 카페가 많아, 음료와 디저트를 동시에 즐기기에 제격입니다."),
    ("서울 강남구에 유명 카페 있어?",
    "서울 강남구에는 분위기와 맛을 모두 갖춘 유명 카페들이 많으며, 그중에서도 타짜도르(Tazza d'Oro)는 이탈리아 정통 에스프레소를 즐길 수 있는 곳으로, 진한 커피 맛과 차분한 인테리어가 인상적입니다. 또한 트리오드(TRIODE)는 퍼먹는 스푼 케이크로 유명한 디저트 카페로, 아메리카노와 함께 달콤한 케이크를 즐기기에 제격이며, 테라스와 실내 공간 모두 아늑한 분위기를 자랑합니다. 베이커스트 브라운(BAKEST BROWN)은 3층짜리 단독 건물에 자리한 베이커리 카페로, 계절마다 바뀌는 디저트와 햇살 가득한 창가 자리가 매력적이며, 커피와 함께 다양한 베이커리를 직접 골라 담을 수 있어 만족도가 높습니다. 이 외에도 강남에는 인스타그램에서 핫한 감성 카페들이 많아, 커피 한 잔과 함께 여유로운 시간을 보내기에 좋은 장소들이 즐비해 있습니다."),
    ("커피숍 예약 가능해?",
    "이들 카페는 대부분 예약보다는 시간대를 잘 선택해 방문하는 것이 추천되며, 특별한 이벤트나 단체 방문 시에는 사전 문의를 통해 가능 여부를 확인하는 것이 바람직합니다. 서울 강남구에 위치한 타짜도르(Tazza d'Oro)는 이탈리아 정통 에스프레소를 즐길 수 있는 카페로, 매장이 작아 보이지만 내부에 1.5층 공간이 있어 테이블 수는 제법 많으며, 예약보다는 피크타임을 피해 방문하는 것이 좋습니다.  트리오드(TRIODE)는 퍼먹는 케이크로 유명한 디저트 카페로, 평일 점심 시간에는 자리를 먼저 맡아야 주문이 가능할 정도로 붐비며, 별도의 예약 시스템은 없지만 이른 시간에 방문하면 여유롭게 즐길 수 있습니다.  베이커스트 브라운(BAKEST BROWN)은 단독 건물 3층 전체가 카페로 운영되며, 다양한 베이커리와 음료를 즐길 수 있는 공간이지만 예약보다는 현장 방문이 일반적이며, 햇살 좋은 날에는 테라스 좌석이 인기가 많아 일찍 가는 것이 좋습니다."),
    ("테이크아웃 가능한 메뉴 알려줘.",
    "이들 카페 모두 테이크아웃이 가능하며, 각 매장마다 포장 방식이나 메뉴 구성에 차이가 있으므로 방문 전 확인하면 더욱 만족스러운 이용이 가능합니다. 서울 강남구에 위치한 타짜도르(Tazza d'Oro)는 이탈리아 정통 에스프레소를 전문으로 하는 카페로, 매장 내에서 즐기는 고객이 많지만 테이크아웃도 가능하여 바쁜 출근길에 간편하게 커피를 즐길 수 있습니다.  트리오드(TRIODE)는 퍼먹는 케이크와 다양한 디저트로 유명한 감성 카페로, 대부분의 음료와 디저트가 포장 가능하며, 특히 테이크아웃 컵에 담긴 라떼와 아메리카노가 인기를 끌고 있습니다. 베이커스트 브라운(BAKEST BROWN)은 3층짜리 단독 건물에 자리한 베이커리 카페로, 다양한 베이커리와 음료를 테이크아웃할 수 있으며, 포장 시에도 고급스러운 패키지로 제공되어 선물용으로도 적합합니다."),
]

# 대화 진행 예시 (pairs 리스트를 HumanMessage/AIMessage 쌍으로 변환해 그래프에 주입)
for human_txt, ai_txt in pairs:
    # 한 턴을 HumanMessage와 AIMessage 쌍으로 app.invoke에 보내 대화 기록으로 저장 (dict → Message 객체 변환)
    app.invoke(
        {"messages": [HumanMessage(content=human_txt), AIMessage(content=ai_txt)]},
        config                                                          # 세션 ID 포함 – 히스토리 유지
    )

<small>

* 실행 시간 (1m 47.1s)

In [ ]:
# 최종 요약 + 최근 버퍼 확인
stored = app.get_state(config)
print("=== 최종 요약 ===")
print(stored.values["summary"],"\n")                            # 요약 출력

print("=== 최근 메시지 버퍼 (최근 4개) ===", "\n")
for m in stored.values["messages"]:
    print(m.content)                                            # 최근 메시지 원본 출력

<small>

* 셀 출력

    ```markdown
    === 최종 요약 ===
    각 카페에서 제공하는 테이크아웃 가능한 메뉴는 다음과 같습니다:

    1. **타짜도르 (Tazza d'Oro)**: 이탈리아 정통 에스프레소와 다양한 커피 음료를 테이크아웃할 수 있습니다. 바쁜 출근길에 간편하게 즐길 수 있는 커피가 인기입니다.

    2. **트리오드 (TRIODE)**: 다양한 음료와 함께 퍼먹는 케이크, 디저트가 테이크아웃 가능합니다. 특히 테이크아웃 컵에 담긴 라떼와 아메리카노가 많은 사랑을 받고 있습니다.

    3. **베이커스트 브라운 (BAKEST BROWN)**: 다양한 베이커리와 음료를 테이크아웃할 수 있으며, 포장 시 고급스러운 패키지로 제공되어 선물용으로도 적합합니다.

    각 매장마다 메뉴와 포장 방식이 다를 수 있으니, 방문 전 미리 확인하는 것이 좋습니다.

    === 최근 메시지 버퍼 (최근 4개) ===
    
    테이크아웃 가능한 메뉴 알려줘.
    이들 카페 모두 테이크아웃이 가능하며, 각 매장마다 포장 방식이나 메뉴 구성에 차이가 있으므로 방문 전 확인하면 더욱 만족스러운 이용이 가능합니다. 서울 강남구에 위치한 타짜도르(Tazza d'Oro)는 이탈리아 정통 에스프레소를 전문으로 하는 카페로, 매장 내에서 즐기는 고객이 많지만 테이크아웃도 가능하여 바쁜 출근길에 간편하게 커피를 즐길 수 있습니다.  트리오드(TRIODE)는 퍼먹는 케이크와 다양한 디저트로 유명한 감성 카페로, 대부분의 음료와 디저트가 포장 가능하며, 특히 테이크아웃 컵에 담긴 라떼와 아메리카노가 인기를 끌고 있습니다. 베이커스트 브라운(BAKEST BROWN)은 3층짜리 단독 건물에 자리한 베이커리 카페로, 다양한 베이커리와 음료를 테이크아웃할 수 있으며, 포장 시에도 고급스러운 패키지로 제공되어 선물용으로도 적합합니다.
    ```


---

* *next: 벡터저장소 검색 메모리(VectorStoreRetrieverMemory)*

---